### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=Wg4tV6RSnmQoPVle5JnVGAwnvL5huftZ&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&begin_date=20130101&end_date=20230531&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count'

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0,20):
    
    # create query with a page number
    # API results show 10 articles at a time    
    query_request_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_request_url).json()
    
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in reviews["response"]["docs"]:
            reviews_list.append(article)
            
        # Print the page that was just retrieved
        print(f'successfully retrieved page {page}')
        
    except:
        # Print the page number that had no results then break from the loop
        print(f'exception: page {page} has no results')
        break

successfully retrieved page 0
successfully retrieved page 1
successfully retrieved page 2
successfully retrieved page 3
successfully retrieved page 4
successfully retrieved page 5
successfully retrieved page 6
successfully retrieved page 7
successfully retrieved page 8
successfully retrieved page 9
successfully retrieved page 10
successfully retrieved page 11
successfully retrieved page 12
successfully retrieved page 13
successfully retrieved page 14
successfully retrieved page 15
successfully retrieved page 16
successfully retrieved page 17
successfully retrieved page 18
successfully retrieved page 19


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
reviews_formatted = json.dumps(reviews['response']['docs'][:5], indent=4)
print(reviews_formatted)

[
    {
        "web_url": "https://www.nytimes.com/2017/05/11/movies/hounds-of-love-review.html",
        "snippet": "This feature debut of the Australian filmmaker Ben Young resembles a real-life couple\u2019s killing spree in the 1980s.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: \u2018Hounds of Love\u2019 Is Tense and Deadly Down Under",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Hounds of Love",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Hounds of Love (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
     

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = pd.json_normalize(reviews_list)
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_reviews_df['title'] = nyt_reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df['keywords'] = nyt_reviews_df['keywords'].apply(extract_keywords)
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = nyt_reviews_df['title'].tolist()
titles_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print('Wait for it...')
        
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    query_url = f'{url}{title}{tmdb_key_string}'
    movie_response = requests.get(query_url).json()
    
    # Include a try clause to search for the full movie details.
    try:
            
        # Get movie id
        for id in movie_response['results']:
            movie_id = id['id']
            
        # Make a request for a the full movie details
        movie_query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
        
        # Execute "GET" request with url
        movie_details = requests.get(movie_query_url).json()

        # Extract the genre names into a list 
        genres = [genre['name'] for genre in movie_details['genres']]
            
        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
            
        # Extract the production_countries' name into a list
        production_countries = [origin['name'] for origin in movie_details['production_countries']]

        # Add the relevant data to a dictionary
        relevant_dictionary = {
            'title': movie_details['title'],
            'original_title': movie_details['original_title'],
            'budget': movie_details['budget'],
            'language': movie_details['original_language'],
            'homepage': movie_details['homepage'],
            'overview': movie_details['overview'],
            'popularity': movie_details['popularity'],
            'runtime': movie_details['runtime'],
            'revenue': movie_details['revenue'],
            'release_date': movie_details['release_date'],
            'vote_average': movie_details['vote_average'],
            'vote_count': movie_details['vote_count'],
            'genre': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(relevant_dictionary)
        
        # Print out the title that was found
        print(relevant_dictionary['title'])
        
    # Use the except clause to print out a statement if a movie
    # is not found.
    except:
        print('movie not found')
        break

The Attachment Diaries
The Attachment Diaries
You Can Live Forever
A Tourist's Guide to Love
Other People's Children
One True Love
The Lost Weekend: A Love Story
One in a Thousand
Your Place... or Mine?
Love in the Time of Fentanyl
Pamela, A Love Story
She Comes From the Other Side of the Water
After (A Love Story)
Alcarràs
Nelly Don the Musical Movie
Lady Chatterley's Lover
Sound of Christmas
Inspector Dupin: Brittany's Tide
Bones and All
My Policeman
Fate
Waiting for Bojangles
My dad never told me I love you
Lady Snowblood 2: Love Song of Vengeance
Alone Together
Ovid and the Art of Love
The Color Wheel
Marvel Studios Assembled: The Making of Thor: Love and Thunder
Both Sides of the Blade
Fire Maidens of Outer Space
Crazy Love
Stay Prayed Up
Nightmares
Brett Gelman's Dinner in America
New York Minute
Anaïs in Love
Everybody's All-American
All You See
I Saw Mommy Kissing Santa Claus
Love After Death
Deep Waters
Being the Ricardos
Cyrano de Bergerac
The Residents - In Between Dreams (L

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
formatted_movies_list = json.dumps(tmdb_movies_list[:5], indent=4)
print(formatted_movies_list)

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 3.584,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    

In [13]:
# Convert the results to a DataFrame
movies_list_df = pd.json_normalize(tmdb_movies_list)
movies_list_df = movies_list_df[['title', 'original_title', 'budget', 'genre', 'language', 'spoken_languages', 'homepage', 'overview', 'popularity', 'runtime', 'revenue', 'release_date', 'vote_average', 'vote_count', 'production_countries']].drop_duplicates(subset='title')
movies_list_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",3.584,102,0,2021-10-07,3.000,4,[Argentina]
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",46.324,96,15055,2023-03-24,6.700,33,"[Canada, United States of America]"
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",20.990,96,0,2023-04-21,6.300,158,[United States of America]
4,Other People's Children,Other People's Children,0,[],en,[],,A series of home move style shots of a mother ...,0.600,15,0,1978-01-01,0.000,0,[United States of America]
5,One True Love,Однолюбы,0,[Drama],ru,[Russian],,"The story is about old village residents who, ...",1.340,80,0,1983-07-13,0.000,0,[Soviet Union]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",7.927,93,0,2017-09-15,5.800,40,[]
182,"Cry for Me, Billy","Cry for Me, Billy",0,[Western],en,[English],,Drifter falls in love with Native American gir...,2.353,93,0,1972-08-01,5.000,3,[United States of America]
187,Lost Souvenirs,Souvenirs perdus,0,"[Fantasy, Drama]",fr,[French],,Suppose lost and found objects could talk... B...,5.600,125,0,1950-11-11,5.500,14,[France]
195,L'altra metà del cielo,L'altra metà del cielo,0,"[Comedy, Romance]",it,[Italian],,"Don Vincenzo, a priest sent to a mining villag...",3.601,97,0,1977-03-04,6.200,3,[Italy]


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
movies_merged_df = movies_list_df.merge(nyt_reviews_df, on='title', how='inner')
movies_merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",3.584,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",46.324,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.031,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
3,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,1.032,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
4,Alcarràs,Alcarràs,0,[Drama],ca,[Catalan],https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",17.345,120,...,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None
5,Bones and All,Bones and All,18000000,"[Drama, Horror, Romance]",it,[English],https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",24.723,131,...,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
6,My Policeman,My Policeman,0,"[Drama, Romance]",en,[English],https://www.amazon.com/dp/B09Y8SHDPD,"In the late 1990s, the arrival of elderly inva...",16.763,113,...,"‘My Policeman’ Review: Two Love Affairs, Equal...",None,None,My Policeman,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
7,Waiting for Bojangles,En attendant Bojangles,0,"[Drama, Comedy, Romance]",fr,"[English, French, Spanish]",,A boy and his eccentric parents leave their ho...,14.421,124,...,‘Waiting for Bojangles’ Review: Endless Love,None,None,Waiting for Bojangles,None,None,None,By Calum Marsh,"[{'firstname': 'Calum', 'middlename': None, 'l...",None
8,Alone Together,Sami zajedno,0,[Drama],sr,[Serbian],,"Returning home to Belgrade, under the strange ...",0.600,21,...,‘Alone Together’ Review: Finding Love in a Pan...,None,None,Alone Together,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
9,Both Sides of the Blade,Avec amour et acharnement,0,"[Drama, Romance]",fr,[French],https://www.bothsidesoftheblade.com/,Jean and Sara have been living together for 10...,10.575,117,...,‘Both Sides of the Blade’ Review: Who Do You L...,None,None,A Bliss Doomed To Be Doused,None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
bracket_removal = ['genre', 'keywords', 'spoken_languages']

# Create a list of characters to remove
char_bye_bye = ['[', ']', "'", '"']

# Loop through the list of columns to fix
for column in bracket_removal:
    
    # Convert the column to type 'str'
    movies_merged_df[column] = movies_merged_df[column].astype(str)

    # Loop through characters to remove
    for char in char_bye_bye:
        movies_merged_df[column] = movies_merged_df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
movies_merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",3.584,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",46.324,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.031,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
3,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,Documentary,en,English,,An intimate portrait of a community fighting t...,1.032,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
4,Alcarràs,Alcarràs,0,Drama,ca,Catalan,https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",17.345,120,...,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None
5,Bones and All,Bones and All,18000000,"Drama, Horror, Romance",it,English,https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",24.723,131,...,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
6,My Policeman,My Policeman,0,"Drama, Romance",en,English,https://www.amazon.com/dp/B09Y8SHDPD,"In the late 1990s, the arrival of elderly inva...",16.763,113,...,"‘My Policeman’ Review: Two Love Affairs, Equal...",None,None,My Policeman,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
7,Waiting for Bojangles,En attendant Bojangles,0,"Drama, Comedy, Romance",fr,"English, French, Spanish",,A boy and his eccentric parents leave their ho...,14.421,124,...,‘Waiting for Bojangles’ Review: Endless Love,None,None,Waiting for Bojangles,None,None,None,By Calum Marsh,"[{'firstname': 'Calum', 'middlename': None, 'l...",None
8,Alone Together,Sami zajedno,0,Drama,sr,Serbian,,"Returning home to Belgrade, under the strange ...",0.600,21,...,‘Alone Together’ Review: Finding Love in a Pan...,None,None,Alone Together,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
9,Both Sides of the Blade,Avec amour et acharnement,0,"Drama, Romance",fr,French,https://www.bothsidesoftheblade.com/,Jean and Sara have been living together for 10...,10.575,117,...,‘Both Sides of the Blade’ Review: Who Do You L...,None,None,A Bliss Doomed To Be Doused,None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None


In [16]:
# Drop "byline.person" column
movies_merged_df = movies_merged_df.drop(columns=['byline.person'])


In [17]:
# Delete duplicate rows and reset index
merged_cleaned_df = movies_merged_df.drop_duplicates(subset=['title']).reset_index(drop=True)
merged_cleaned_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",3.584,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",46.324,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.031,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
3,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,Documentary,en,English,,An intimate portrait of a community fighting t...,1.032,85,...,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None
4,Alcarràs,Alcarràs,0,Drama,ca,Catalan,https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",17.345,120,...,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,None


In [18]:
# Export data to CSV without the index
merged_cleaned_df.to_csv('output_csv.csv', index=False)